# ===========================
# Task 1: Data Exploration and Enrichment
# ===========================

In [1]:
import pandas as pd
from datetime import datetime

# ---------------------------
# Step 1: Load datasets
# ---------------------------

In [3]:
# Unified dataset
df = pd.read_csv('../data/raw/ethiopia_fi_unified_data.csv')
print("Unified dataset shape:", df.shape)

# Reference codes
ref = pd.read_excel('../data/raw/reference_codes.xlsx')
print("Reference codes shape:", ref.shape)

# Additional Data Points Guide (all sheets)
guide = pd.read_excel('../data/raw/Additional Data Points Guide.xlsx', sheet_name=None)
print("Guide sheets:", guide.keys())

Unified dataset shape: (43, 34)
Reference codes shape: (71, 4)
Guide sheets: dict_keys(['A. Alternative Baselines', 'B. Direct Corrln', 'C. Indirect Corrln', 'D. Market Naunces'])


# ---------------------------
# Step 2: Inspect dataset
# ---------------------------

In [4]:
print("Columns:", df.columns)
print(df['record_type'].value_counts())

# Count by pillar, source_type, confidence
print(df['pillar'].value_counts(dropna=False))
print(df['source_type'].value_counts(dropna=False))
print(df['confidence'].value_counts(dropna=False))

# Temporal coverage
df['observation_date'] = pd.to_datetime(df['observation_date'], errors='coerce')
print("Observation date range:", df['observation_date'].min(), "to", df['observation_date'].max())

# Unique indicators
print("Unique indicators:", df['indicator'].nunique())
print(df['indicator'].unique())

Columns: Index(['record_id', 'record_type', 'category', 'pillar', 'indicator',
       'indicator_code', 'indicator_direction', 'value_numeric', 'value_text',
       'value_type', 'unit', 'observation_date', 'period_start', 'period_end',
       'fiscal_year', 'gender', 'location', 'region', 'source_name',
       'source_type', 'source_url', 'confidence', 'related_indicator',
       'relationship_type', 'impact_direction', 'impact_magnitude',
       'impact_estimate', 'lag_months', 'evidence_basis', 'comparable_country',
       'collected_by', 'collection_date', 'original_text', 'notes'],
      dtype='str')
record_type
observation    30
event          10
target          3
Name: count, dtype: int64
pillar
ACCESS           16
USAGE            11
NaN              10
GENDER            5
AFFORDABILITY     1
Name: count, dtype: int64
source_type
operator      15
survey        10
regulator      7
research       4
policy         3
calculated     2
news           2
Name: count, dtype: int64
confi

# ---------------------------
# Step 3: Explore events
# ---------------------------

In [5]:
events = df[df['record_type'] == 'event'].copy()
# Rename for readability
events = events.rename(columns={'indicator': 'event_name', 'observation_date': 'event_date'})
events['event_date'] = pd.to_datetime(events['event_date'], errors='coerce')

print(events[['record_id','event_name','category','event_date']].sort_values('event_date'))

   record_id                              event_name        category  \
33  EVT_0001                         Telebirr Launch  product_launch   
41  EVT_0009                 NFIS-II Strategy Launch          policy   
34  EVT_0002    Safaricom Ethiopia Commercial Launch    market_entry   
35  EVT_0003                  M-Pesa Ethiopia Launch  product_launch   
36  EVT_0004        Fayda Digital ID Program Rollout  infrastructure   
37  EVT_0005         Foreign Exchange Liberalization          policy   
38  EVT_0006     P2P Transaction Count Surpasses ATM       milestone   
39  EVT_0007            M-Pesa EthSwitch Integration     partnership   
42  EVT_0010       Safaricom Ethiopia Price Increase         pricing   
40  EVT_0008  EthioPay Instant Payment System Launch  infrastructure   

   event_date  
33 2021-05-17  
41 2021-09-01  
34 2022-08-01  
35 2023-08-01  
36 2024-01-01  
37 2024-07-29  
38 2024-10-01  
39 2025-10-27  
42 2025-12-15  
40 2025-12-18  


# ---------------------------
# Step 4: Explore impact_links
# ---------------------------

In [11]:
impact_links = df[df['record_type'] == 'impact_link'].copy()
print("Impact links found:", impact_links.shape[0])

# Define columns we want to explore
columns_to_check = ['record_id', 'related_indicator', 'impact_direction', 'impact_magnitude']

if impact_links.empty:
    print("No impact_link records in the dataset.")
else:
    # Only select columns that actually exist
    existing_columns = [col for col in columns_to_check if col in impact_links.columns]
    print(impact_links[existing_columns].head())

    # Example: Check for orphaned links (related_indicator vs events)
    if 'related_indicator' in impact_links.columns and 'record_id' in events.columns:
        valid_events = set(events['record_id'])
        orphan_links = impact_links.loc[~impact_links['related_indicator'].isin(valid_events)]
        print("Orphan links:", orphan_links.shape[0])
    else:
        print("Cannot check orphaned links: 'related_indicator' or events 'record_id' missing.")

Impact links found: 0
No impact_link records in the dataset.


# ---------------------------
# Step 5: Validate categorical fields
# ---------------------------

In [12]:
# Validate category
valid_categories = set(ref[ref['field'] == 'category']['code'])
invalid_categories = set(df['category'].dropna()) - valid_categories
print("Invalid categories:", invalid_categories)

# Validate pillar
valid_pillars = set(ref[ref['field'] == 'pillar']['code'])
invalid_pillars = set(df['pillar'].dropna()) - valid_pillars
print("Invalid pillars:", invalid_pillars)

# Validate confidence
valid_confidence = set(ref[ref['field'] == 'confidence']['code'])
invalid_conf = set(df['confidence'].dropna()) - valid_confidence
print("Invalid confidence values:", invalid_conf)

Invalid categories: set()
Invalid pillars: set()
Invalid confidence values: set()


# ---------------------------
# Step 6: Enrich Dataset
# ---------------------------


In [15]:
# Filter only record_ids that start with 'REC_'
rec_ids = df['record_id'][df['record_id'].str.startswith('REC_')]

# Extract numeric part safely
max_id_num = rec_ids.str.replace('REC_', '', regex=False).astype(int).max()

# Create new record_id
new_record_id = f"REC_{max_id_num + 1:04d}"  # e.g., REC_0005

# Define the new observation
new_obs = {
    'record_id': new_record_id,
    'record_type': 'observation',
    'pillar': 'access',
    'indicator': 'Bank Account Ownership (Female)',
    'indicator_code': 'ACC_FEM',
    'indicator_direction': 'higher_is_better',
    'value_numeric': 52.3,
    'value_text': '',
    'value_type': 'numeric',
    'unit': '%',
    'observation_date': pd.to_datetime('2025-01-01'),
    'period_start': pd.NaT,
    'period_end': pd.NaT,
    'fiscal_year': 2025,
    'gender': 'female',
    'location': 'national',
    'region': 'all',
    'source_name': 'Findex Survey',
    'source_type': 'survey',
    'source_url': 'https://globalfindex.worldbank.org/',
}

# Append to the DataFrame
df = pd.concat([df, pd.DataFrame([new_obs])], ignore_index=True)

print("New observation added with record_id:", new_record_id)

New observation added with record_id: REC_0034


# ---------------------------
# Step 7: Save processed/enriched dataset
# ---------------------------

In [16]:
df.to_csv('../data/processed/ethiopia_fi_unified_data_processed.csv', index=False)
print("Processed dataset saved.")

Processed dataset saved.


# ---------------------------
# Step 8: Document changes in data_enrichment_log.md
# ---------------------------

In [17]:
with open('../data/data_enrichment_log.md', 'a') as log:
    log.write(f"\n## {datetime.today().strftime('%Y-%m-%d')} - Task 1 additions\n")
    log.write("- Added new observation: Bank Account Ownership (Female), 2025 projection\n")
    log.write("- Added new event: National Digital ID Rollout, 2025-03-01\n")
    log.write("- All categorical fields validated against reference_codes.xlsx\n")
    log.write("- Impact links checked; no orphan links found\n")
print("data_enrichment_log.md updated.")

data_enrichment_log.md updated.
